# Исследование надёжности заёмщиков 

**Заказчик** — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение, количество детей клиента, уровень дохода и цель кредита на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## План работы
- открыть файл и изучить общую информацию;
- обработать пропуски;
- заменить типы данных, где необходимо;
- обработать дубликаты;
- сделать лемматизацию данных;
- сделать категоризацию данных;
- провести анализ данных, ответить на поставленные вопросы;
- сделать общий вывод.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
#Загрузим библиотеки
import pandas as pd
from pymystem3 import Mystem

In [2]:
#Загрузим данные в датафрейм, посмотрим на первые строки и общую информацию
clients = pd.read_csv('/datasets/data.csv')
clients.info()
clients.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#Учитывая небольшой объем данных, исследуем уникальные значения внутри столбцов для лучшего их понимания
for col in clients.columns:
    col_name = col
    values = clients[col].unique()
    print(col)
    print(values)
    print()

children
[ 1  0  3  2 -1  4 20  5]

days_employed
[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]

dob_years
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

education
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

education_id
[0 1 2 3 4]

family_status
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

family_status_id
[0 1 2 3 4]

gender
['F' 'M' 'XNA']

income_type
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

debt
[0 1]

total_income
[253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244093.05050043  82047.41889948]

p

In [4]:
#Посмотрим, как соотносятся столбцы education и family_status с их id, чтобы понять, как разделить значения на категории
print('Категории уровня образования клиента')
for i in range(5):
    print(i, clients[clients['education_id']==i]['education'].unique())

print() 

print('Категории семейного статуса клиента')
for i in range(5):
    print(i, clients[clients['family_status_id']==i]['family_status'].unique())

Категории уровня образования клиента
0 ['высшее' 'ВЫСШЕЕ' 'Высшее']
1 ['среднее' 'Среднее' 'СРЕДНЕЕ']
2 ['неоконченное высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее']
3 ['начальное' 'НАЧАЛЬНОЕ' 'Начальное']
4 ['Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ' 'ученая степень']

Категории семейного статуса клиента
0 ['женат / замужем']
1 ['гражданский брак']
2 ['вдовец / вдова']
3 ['в разводе']
4 ['Не женат / не замужем']


In [5]:
#Уберем ненужные столбцы и переименуем столбец с возрастом клиента
clients = clients.drop(['education_id', 'family_status_id'], axis=1)
clients.rename(columns={'dob_years':'client_age'}, inplace=True)

#Проверим
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 10 columns):
children         21525 non-null int64
days_employed    19351 non-null float64
client_age       21525 non-null int64
education        21525 non-null object
family_status    21525 non-null object
gender           21525 non-null object
income_type      21525 non-null object
debt             21525 non-null int64
total_income     19351 non-null float64
purpose          21525 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 1.6+ MB


### Вывод

Предварительный взгляд на данные показывает следующее:
- **отсутствующие данные** находятся в столбцах **days_employed** и **total_income** и составляют примерно 10% от общего количества строк. То, что данные пропущены одновременно в обоих столбцах, позволяет думать, что клиент не предоставил документ с места работы, соответственно не были введены стаж и ежемесячный доход;
- в столбце **children** встречается отрицательное значение, которое можно посчитать технической ошибкой и взять значение по модулю. Также маловероятно, что у кого-то 20 детей (учитывая, что остальные значения находятся в пределах 0-5), поэтому можно посчитать это ошибкой ввода "2";           
- в столбце **days_employed** встречается много отрицательных значений, кроме того, некоторые значения очень большие и, будучи переведенными в количество лет, неочевидно соотносятся с возрастом человека. Например, трудовой стаж может превышать возраст человека. У нас не стоит задачи анализировать связь между продолжительностью трудоустройства и погашением кредита в срок, следовательно можно заполнить отсутствующие значения медианой по типам дохода для сохранения общей картины. Если бы этот столбец был нужен для анализа, необходимо было бы обращаться к человеку, собиравшему данные для разъяснения значений столбца; 
- название столбца **dob_years** недостаточно ясно отображает его содержание (возраст клиента) - его надо переименовать в client_age. Кроме того, надо будет заменить нулевые значения данного столбца на среднее;
- столбцы **education, education_id** связаны между собой, мы видим, что всего 5 уровней образования клиента. Данные в столбце **education** надо привести к одному виду (нижний регистр) и поменять их тип на category, **education_id** можно удалить, он больше не нужен;
- столбцы **family_status, family_status_id** также связаны между собой, мы видим, что всего 5 уровней семейного положения клиента. Данные в столбце **family_status** надо поменять на тип category, **family_status_id** можно удалить, он больше не нужен;
- в столбце **gender** встречаются пропущенные значения под видом 'XNA', их надо заменить и поменять тип на category;  
- в столбце **income_type** надо поменять тип на category;
- тип данных столбца **debt** надо поменять на bool, так как это позволит сэкономить место, но в то же время даст возможность проводить математические операции со значениями;
- в **столбце total_income** необходимо заполнить пропуски средними значениями, например, медианой по типу дохода. Значения самого столбца можно перевести в тип int, чтобы зрительно было проще с ними работать; 
- в столбце **purpose** надо сделать лемматизацию и выделить основные группы назначения кредита, чтобы потом использовать их для группировки данных.  

Также для дальнейшего анализа надо будет добавить столбцы, позволяющие объединять клиентов в группы по определенному признаку:
- наличие/ отсутствие детей - **children_status**
- уровень дохода - **income_status**
- главная цель кредита - **purpose_category**

### Шаг 2. Предобработка данных

### Обработка пропусков

#### Столбец days_employed

In [6]:
#Посмотрим количество отсутствующих значений в столбце 
clients['days_employed'].isnull().sum()

2174

In [7]:
#Сгруппируем данные и заполним отсутствующие данные медианой по группам
clients['days_employed'] = clients.groupby(['income_type']).days_employed.apply(lambda x: x.fillna(x.median()))

#Проверим, есть ли отсутствующие значения
clients['days_employed'].isnull().sum()

0

#### Столбец total_income

In [8]:
#Посмотрим количество отсутствующих значений в столбце 
clients['total_income'].isnull().sum()

2174

In [9]:
#Выделим и сохраним индексы тех строк, где встречаются пропущенные значения, чтобы потом проверить, что они заполнились
nan_index = clients.index[clients['total_income'].isnull()]
nan_index

Int64Index([   12,    26,    29,    41,    55,    65,    67,    72,    82,
               83,
            ...
            21415, 21423, 21426, 21432, 21463, 21489, 21495, 21497, 21502,
            21510],
           dtype='int64', length=2174)

In [10]:
#Проверим, что работает
clients.iloc[nan_index].head(10)

,children,days_employed,client_age,education,family_status,gender,income_type,debt,total_income,purpose
12,0,365213.306266,65,среднее,гражданский брак,M,пенсионер,0,NaN,сыграть свадьбу
26,0,-2689.368353,41,среднее,женат / замужем,M,госслужащий,0,NaN,образование
29,0,365213.306266,63,среднее,Не женат / не замужем,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,-2689.368353,50,среднее,женат / замужем,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,365213.306266,54,среднее,гражданский брак,F,пенсионер,1,NaN,сыграть свадьбу
65,0,-1547.382223,21,среднее,Не женат / не замужем,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,365213.306266,52,высшее,женат / замужем,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,-2689.368353,32,высшее,женат / замужем,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,-1574.202821,50,высшее,женат / замужем,F,сотрудник,0,NaN,жилье
83,0,-1574.202821,52,среднее,женат / замужем,M,сотрудник,0,NaN,жилье


In [11]:
#Определим, в каких группах дохода есть отсутствующие значения
income_type_with_nan = clients.iloc[nan_index].groupby('income_type').groups.keys()
income_type_with_nan

dict_keys(['госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник'])

In [12]:
#Посмотрим базовую статистку по этим типам доходов
clients.query('income_type in @income_type_with_nan').groupby('income_type')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
госслужащий,1312.0,170898.309923,96878.763554,29200.077193,105294.668418,150447.935283,209173.052434,9.104515e+05
компаньон,4577.0,202417.461462,130481.095793,28702.812889,125887.745820,172357.950966,243907.839199,2.265604e+06
пенсионер,3443.0,137127.465690,80246.953231,20667.263793,82881.443465,118514.486412,169700.433010,7.351033e+05
предприниматель,1.0,499163.144947,NaN,499163.144947,499163.144947,499163.144947,499163.144947,4.991631e+05
сотрудник,10014.0,161380.260488,91322.514795,21367.648356,102795.633631,142594.396847,196828.082600,1.726276e+06


Базовая статистика показывает, что разброс от минимального до максимального значения в интересующих нас группах достаточно велик, следовательно, нам лучше взять медиану, чем среднее, и ею заполнить отсутствующие значения.

In [13]:
#Сохраним медиану по группам до заполнения пропусков для дальнейшей проверки
median_before_fillna = clients.query('income_type in @income_type_with_nan').groupby('income_type')['total_income'].median()
median_before_fillna

income_type
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
Name: total_income, dtype: float64

In [14]:
#Сгруппируем данные и заполним отсутствующие данные медианой по группам
clients['total_income'] = clients.groupby(['income_type']).total_income.apply(lambda x: x.fillna(x.median()))

#Проверим, есть ли отсутствующие значения
clients['total_income'].isnull().sum()

0

In [15]:
#Проверим, заполнились ли строки, в которых до этого отсутствовало значение, медианами по группам доходов
clients.iloc[nan_index].groupby('income_type')['total_income'].median() == median_before_fillna

income_type
госслужащий        True
компаньон          True
пенсионер          True
предприниматель    True
сотрудник          True
Name: total_income, dtype: bool

#### Столбец gender

In [16]:
#Исследуем значения в столбце gender
print(clients.gender.value_counts())

#Найдем клиентов с неопределенным полом
print(clients[clients.gender=='XNA'])

#Заменим его на F
clients.loc[clients['gender'] == 'XNA', 'gender'] = 'F'

#Проверим 
clients.gender.value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64
       children  days_employed  client_age            education  \
10701         0   -2358.600502          24  неоконченное высшее   

          family_status gender income_type  debt   total_income  \
10701  гражданский брак    XNA   компаньон     0  203905.157261   

                    purpose  
10701  покупка недвижимости  


F    14237
M     7288
Name: gender, dtype: int64

#### Столбец client_age

Пойдем тем же путем, что при заполнении отсутствующих значений столбца total_income - сгруппируем по типу доходов и найдем медиану возраста. Выберем медиану, потому что возраст как правило имеет нормальное распределение, соответственно, среднее и медиана будут примерно одинаковы, но медиана будет целым числом, что нам и надо. 

In [17]:
#Посчитаем, у какого количества клиентов не проставлен возраст
(clients['client_age']==0).sum()

101

In [18]:
#Выделим и сохраним индексы тех строк, где встречаются пропущенные значения, чтобы потом проверить, что они заполнились
nan_index_age = clients.index[clients['client_age']==0]
nan_index_age

Int64Index([   99,   149,   270,   578,  1040,  1149,  1175,  1386,  1890,
             1898,
            ...
            18539, 18732, 18851, 19116, 19371, 19829, 20462, 20577, 21179,
            21313],
           dtype='int64', length=101)

In [19]:
#Проверим, что работает
clients.iloc[nan_index_age].head(10)

,children,days_employed,client_age,education,family_status,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,женат / замужем,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,в разводе,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,женат / замужем,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,женат / замужем,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,в разводе,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,женат / замужем,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,женат / замужем,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,женат / замужем,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,-1574.202821,0,высшее,Не женат / не замужем,F,сотрудник,0,142594.396847,жилье
1898,0,370144.537021,0,среднее,вдовец / вдова,F,пенсионер,0,127400.268338,на покупку автомобиля


In [20]:
#Определим, в каких группах дохода есть отсутствующие значения возраста
income_type_with_nan_age = clients.iloc[nan_index_age].groupby('income_type').groups.keys()
income_type_with_nan_age

dict_keys(['госслужащий', 'компаньон', 'пенсионер', 'сотрудник'])

In [21]:
#Сохраним медиану по группам до заполнения пропусков
median_before_fillna_age = clients.query('income_type in @income_type_with_nan_age').groupby('income_type')['client_age'].median()
median_before_fillna_age

income_type
госслужащий    40
компаньон      39
пенсионер      60
сотрудник      39
Name: client_age, dtype: int64

In [22]:
#Сгруппируем данные и заполним отсутствующие данные медианой по группам
clients['client_age'] = clients.groupby(['income_type'])['client_age'].apply(lambda x: x.replace(0, x.median()))

#Проверим
(clients['client_age']==0).sum()
clients.iloc[nan_index_age].head(10)

,children,days_employed,client_age,education,family_status,gender,income_type,debt,total_income,purpose
99,0,346541.618895,60,Среднее,женат / замужем,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,39,среднее,в разводе,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,39,среднее,женат / замужем,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,60,среднее,женат / замужем,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,39,высшее,в разводе,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,39,среднее,женат / замужем,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,60,среднее,женат / замужем,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,40,высшее,женат / замужем,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,-1574.202821,39,высшее,Не женат / не замужем,F,сотрудник,0,142594.396847,жилье
1898,0,370144.537021,60,среднее,вдовец / вдова,F,пенсионер,0,127400.268338,на покупку автомобиля


In [ ]:
#Проверим, заполнились ли строки, в которых до этого отсутствовало значение, медианами по группам доходов
clients.iloc[nan_index_age].groupby('income_type')['client_age'].median() == median_before_fillna_age

### Вывод

2174 отсутствующих значения в столбце **total_income**  и **days_employed** были заполнены медианами по типу дохода. Медиана была выбрана исходя из большого разброса между минимальными и максимальными значениями. Для заполнения медианами был выбран метод apply() в сочетании с lambda и replace(). 

Для проверки заполнения значений **total_income** были выделены индексы строк, где они изначально отсутствовали данные по доходу, и затем медианы дохода данных строк, сгруппированные по типам доходов, были сравнены с медианами дохода всех данных, сгруппированных по типам дохода. 

Что касается столбца **gender**, пропущенное значение только одно. Можно предположить, что это была техническая неточность. Посмотрев на общее распределение данные по полу (большинство клиентов женского пола) и на данные строки с пропущенным значением, было решено подставить его как F, что представляется вполне логичным (возраст 24 года, источник заработка - компаньон, данных по трудовому стажу нет). В любом случае одно значения не окажет большого влияния на общий анализ. 

В столбце **client_age** было найдено 101 значение равное 0. Отсутствующие значения были заменены по принципу, примененному к столбцу total_income - медианами по типу дохода. Медиана была выбрана, так как в отличие от среднего она представляет собой целое число. Для проверки заполнения значений также были выделены индексы строк, где они изначально отсутствовал возраст, и затем медианы значений возраста строк, сгруппированные по типам доходов, были сравнены с медианами возраста всех данных, сгруппированных по типам дохода. 

In [ ]:
#Проверим, что отсутствующих значений больше нет
clients.info()

### Замена типа данных

#### Столбец children

In [ ]:
#Уберем отрицательные значения в столбце и заменим 20 на 2
clients['children'] = abs(clients.children)
clients.loc[clients['children'] == 20, 'children'] = 2

#Проверим содержимое нового столбца и посмотрим, как распределяются цели кредита
clients.children.value_counts(normalize=True)

#### Столбец education и замена типов данных

In [ ]:
#Переведем значение в нижний регистр, чтобы они были одинаковые
clients['education'] = clients['education'].str.lower()

#Заменим типы данных для экономии памяти
clients = clients.astype({'education':'category', 
                'family_status':'category', 
                'gender':'category', 
                'income_type':'category', 
                'debt':'bool', 
                'total_income': int})

#Посмотрим использование памяти и новые типы данных
clients.info()

### Вывод

В столбце children были убраны отрицательные значения и заменены значения "20" на "2" как более логичные. В столбце education все значения привели к нижнему регистру, благодаря чему осталось только 5 основных значений, которые были переделаны в категории. 

Также типы данных столбцов family_status, gender, income_type были заменены на категориальные, debt на булевые, total_income на целочисленный. Использован метод astype(), который принимает словарь и позволяет сделать сразу разные замены для разных столбцов. 
Объем занимаемой памяти уменьшился с 2.0+ MB до 946.9+ KB

### Обработка дубликатов

In [ ]:
#Посмотрим на количество дубликатов и уберем их. Проверим, что это отразилось в clients
print('Количество дубликатов: ', clients.duplicated().sum())
clients = clients.drop_duplicates().reset_index(drop=True)
clients.duplicated().sum()

### Вывод

Для поиска и удаления дубликатов использованы методы duplicated() в сочетании с sum() и drop_duplicates(), потому что нам надо избавиться от повторяющихся строк, но не повторяющихся значений в столбцах, т.к. они представляют собой категории. Все было найдено 74 дубликата

### Лемматизация

In [ ]:
#Лемматизируем столбец purpose и создадим новым, в котором поместим вылеленные леммы
m = Mystem()
clients['purpose_lemma'] = clients['purpose'].apply(lambda x: m.lemmatize(x))

#Проверим, как прошла лемматизация
clients['purpose_lemma'].head(10)

### Вывод

Предварительный осмотр данных столбца purpose с помощью метода value_counts() показал, что основных можно выделить 5 категорий целей, на которые берут кредит:
- жилье,
- недвижимость,
- автомобиль,
- свадьба,
- образование.

Для того, чтобы создать новый столбец и присвоить ему соответствующую категорию, на столбце purpose была сделана лемматизация. 

### Категоризация данных

#### Категории целей кредитов

In [ ]:
#Создадим и применим функцию для создания нового столбца, отражающего основные цели кредита
def purpose_cats(col):
    '''
    Функция принимает значение столбца и возвращает значение для нового столбца, 
    определяя, какое из ключевых слов находится в исходном столбце 
    '''
    if 'жилье' in col:
        return 'жилье'
    elif 'недвижимость' in col:
        return 'недвижимость'
    elif 'свадьба' in col:
        return 'свадьба'
    elif 'автомобиль' in col:
        return 'автомобиль'
    else:
        return 'образование'

clients['purpose_category'] = clients['purpose_lemma'].apply(purpose_cats).astype('category')

#Проверим содержимое нового столбца и посмотрим, как распределяются цели кредита
clients.purpose_category.value_counts(normalize=True)    

#### Категории доходов 

In [ ]:
#Посмотрим базовую статистику по столбцу total_income
clients['total_income'].describe()

In [ ]:
#Учитывая большой разброс данных, посмотрим, как они рапределяются
clients['total_income'].quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99])

Мы видим, что 99% клиентов имеют доход ниже 505067.84, следовательно, можно посчитать значения выше выбросами и объединить таких клиентов в группу "очень высокий доход". 
Тогда возьмем 99% данных и разделим их по квартилям.

In [ ]:
#Сохраним значение на 99 персентиле 
client_1_percentile = clients['total_income'].quantile(.99)
client_1_percentile

In [ ]:
#Выделим значения квантилей на 99% данных и сохраним в Series
clients_99_quantiles = clients[clients['total_income'] < client_1_percentile]['total_income'].quantile([.25, .5, .75])
clients_99_quantiles

In [ ]:
#Используем полученные значения и создадим категории для уровня дохода
def define_status(col):
    '''
    Функция принимает значение столбца и сравнивает его с определенными рамками значений
    '''
    if col <= clients_99_quantiles.iloc[0]:
        return 'ниже среднего'
    elif clients_99_quantiles.iloc[0] < col <= clients_99_quantiles.iloc[1]:
        return 'средний'
    elif clients_99_quantiles.iloc[1] < col <= clients_99_quantiles.iloc[2]:
        return 'выше среднего'
    elif clients_99_quantiles.iloc[2] < col <= client_1_percentile:
        return 'высокий'
    elif col>client_1_percentile:
        return 'очень высокий'

clients['income_status'] = clients.total_income.apply(define_status).astype('category')

#Проверим содержимое нового столбца и посмотрим, как распределяются доходы
clients.income_status.value_counts(normalize=True)  

#### Категории наличия/ отсутствия детей

In [ ]:
#Создадим категории для наличия/отсутствия детей
def has_children(col):
    '''
    Функция принимает значение столбца и возвращает значение для нового столбца:
    "есть дети", если значение исходного столбца больше 0, и "нет детей", если оно равно 0
    '''
    if col == 0:
        return 'нет детей'
    else:
        return 'есть дети'

clients['children_status'] = clients.children.apply(has_children).astype('category')

#Проверим содержимое нового столбца и посмотрим, как распределяются клиенты на тех, у кого есть дети и у кого их нет
clients.children_status.value_counts(normalize=True)

### Вывод

С помощью функции были выявлены ключевые слова в столбце purpose и создан новый с категориями с соответствующим типом целей кредита **purpose_category**.

Была проанализирована базовая статистика столбца total_income, которая показала большой разброс данных (примерно от 20 000 до 2 000 000) и что 99% клиентов имеют доход ниже примерно 505 000. Соответственно, значения выше было решено объявить выбросами и объединить в отдельную категорию "очень высокий доход". Оставшиеся данные разделили на 4 равные группы, взял за основу для разделения значения квантилей на 25%, 50% и 75%. Для категорий был создан новый столбец **income_status**.

Еще один столбец был создан для того, чтобы можно было сгруппировать клиентов, у которых есть дети, и у кого их нет - **children_status**.

Для создания столбцов были написаны функции для обработки значений столбцов и использован метод apply().

### Шаг 3. Анализ данных

Для анализа данных воспользуемся сводными таблицами, которые можно получить с помощью метода pivot_table(). Учитывая, что значения столбца debt - это только 1 или 0, среднее покажет вероятность наличия задолженности по кредиту, поэтому сводные таблицы будем агрегировать по mean. Для приведения сводных таблиц в более читабельный вид и сортировки данных в них воспользуемся методами melt() и solt_values().

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [ ]:
#Сделаем сводную таблицу по наличию/отсутсвию детей и наличию/отсутствию задолженности по кредиту; посчитаем по среднему
children_debt = clients.pivot_table(values='debt', columns='children_status', aggfunc='mean')

#Переведем ее в более удобный вид и отсортируем значения по убыванию 
children_debt = children_debt.melt(var_name='Наличие детей', value_name='Вероятность долга')\
.sort_values(by=['Вероятность долга'], ascending=False)

#Сделаем сводную таблицу по количеству детей и наличию/отсутствию задолженности по кредиту; посчитаем по среднему
children_debt_details = clients.pivot_table(values='debt', columns='children', aggfunc='mean')

#Переведем ее в более удобный вид и отсортируем значения по убыванию
children_debt_details = children_debt_details.melt(var_name='Количество детей', value_name='Вероятность долга')\
.sort_values(by=['Вероятность долга'], ascending=False).reset_index(drop=True)

#Выведем обе таблицы
print('Зависимость между наличием детей и возвратом кредита в срок:')
print(children_debt)
print()
print('Зависимость между количеством детей и возвратом кредита в срок:')
print(children_debt_details)

### Вывод

Сводная таблица показывает, что у **клиентов с детьми более высокая вероятность иметь задолженность по кредиту**, чем у людей без детей, а именно **на 1.7% больше**. При этом, наибольшая вероятность у тех, у кого четверо детей (9.7%) и двое детей (9.4%), а наименьшая - у кого пятеро (0%). Вероятно, это связано с тем, что с детьми возрастает количество трат и обязательств, особенно если детей много. Интересно, однако, что это не влияет на семьи с 5ю детьми, так как там вообще нет задолженности. 

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [ ]:
#Сделаем сводную таблицу по семейному статусу и наличию/отсутствию задолженности по кредиту; посчитаем по среднему
family_debt = clients.pivot_table(values='debt', columns='family_status', aggfunc='mean')

#Переведем ее в более удобный вид и отсортируем значения по убыванию. 
family_debt = family_debt.melt(var_name='Семейное положение', value_name='Вероятность долга')\
.sort_values(by=['Вероятность долга'], ascending=False).reset_index(drop=True)
print('Зависимость между семейным положением и возвратом кредита в срок')
print(family_debt)

### Вывод

Сводная таблица показывает, что **наибольшая** вероятность иметь задолженность по кредиту **у клиентов, кто не женат/не замужем (9.7%) и кто находится в гражданском браке (9.3%)** . Возможно, это связано с тем, что данная категория клиентов обладает в некотором плане меньшей ответственностью по сравнению с теми, кто находится в браке или в разводе.  **Наименьшая** вероятность - у **вдов/ вдовцов (6.5%)**. Возможно, у них остаются какие-то средства от покойных супругов или в целом меньше обязательств в силу возраста.


- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [ ]:
#Сделаем сводную таблицу по уровню дохода и наличию/отсутствию задолженности по кредиту; посчитаем по среднему
income_debt = clients.pivot_table(values='debt', columns='income_status', aggfunc='mean')

#Переведем ее в более удобный вид и отсортируем значения по убыванию 
income_debt = income_debt.melt(var_name='Уровень дохода', value_name='Вероятность долга')\
.sort_values(by=['Вероятность долга'], ascending=False).reset_index(drop=True)
print('Зависимость между уровнем дохода и возвратом кредита в срок')
print(income_debt)

### Вывод

Сводная таблица показывает, что **наибольшая** вероятность иметь задолженность по кредиту у клиентов **со средним доходом (8.7%)**. Возможно, это связано с тем, что у данной категории клиентов нет достаточного количества накоплений, поэтому какие-то изменения в трудоустройстве или жизненной ситуации могут повлиять на возможность погашения кредита. Чуть ниже вероятность у клиентов с доходом **выше среднего (8.5%)**. Возможно, у них ситуация обратная - есть некоторые накопления, и они могут расплатиться позже и пенни по просроченным выплатам не так важны. **Наименьшая** вероятность - у клиентов **с очень высоким доходом (6.4%)**. Возможно, это связано с тем, что клиенты с таким доходом автоматизируют выплату кредитов в рамках управления финансами.

- **Как разные цели кредита влияют на его возврат в срок?**

In [ ]:
#Сделаем сводную таблицу по целям для кредита и наличию/отсутствию задолженности по кредиту; посчитаем по среднему
purpose_debt = clients.pivot_table(values='debt', columns='purpose_category', aggfunc='mean')

#Переведем ее в более удобный вид и отсортируем значения по убыванию 
purpose_debt = purpose_debt.melt(var_name='Цель кредита', value_name='Вероятность долга')\
.sort_values(by=['Вероятность долга'], ascending=False).reset_index(drop=True)
print('Зависимость между целью кредита и возвратом кредита в срок')
print(purpose_debt)

### Вывод

Здесь сводная таблица показывает, что **наибольшая** вероятность иметь задолженность по кредиту у клиентов, которые берут его на для операций **с автомобилем (9.3%)** и на **образование (9.2%)**. Возможно, это связано с тем, что кредиты на данные цели являются достаточно популярными и воспринимаются как нечто обыденное, где просрочки не так уж нередки или страшны.  **Наименьшая** вероятность - у клиентов, которые берут кредит на операции **с жильем (6.9%)**. Возможно, это связано с тем, что размер таких кредитов значительно выше, на больший срок, и своевременная его выплата для людей стоит в приоритетах.

### Шаг 4. Общий вывод

Были проанализированы данные клиентов банка с целью определить, влияют ли определенные факторы (начилие и количество детей, семейное положеные, уровень доходов и цели кредитов) на погашение кредита в срок. 

После предварительного осмотра данных были:
- определены основные значения столбцов и способы их оптимизации (перевод строк в нижний регистр, изменение типов данных);
- определены столбцы с отсутствующими значениями;
- удалены ненужные столбцы (family_status_id, education_id);
- произведены замены типов данных (education, family_status, gender, income_type - на тип category, debt- на bool, total_income -int);
- переименован столбец с возрастом клиента (из dob_years в client_age).

Далее были заполнены отсутсвующие значения (медианы по типу дохода в столбцах days_employed, total_income, client_age).

Для категоризации данных была произведена лемматизация данных столбца purpose, а также добавлены столбцы: по уровню дохода - income_status, по главной цели кредита - purpose_category, по наличию/ отсутствию детей - children_status.

**Вывод**

Анализ данных показал, что **бОльшая вероятность иметь задолженность по кредиту** у следующих групп клиентов:
- **с детьми** (9.2%), особенно с 4мя (9.7%) или с 2мя (9.4%);
- те, кто **не женат/не замужем** (9.7%) и кто находится **в гражданском браке** (9.3%);
- **со средним доходом** (8.7%) и **выше среднего** (8.5%);
- те, кто берут кредит **для операций с автомобилем** (9.3%) и **на образование** (9.2%);